In [ ]:
# My VS Code has issues connecting to the venv.
# I find that first connecting to one of my conda environments
# and then switching to the venv works.
# I run this cell on conda environment first then switch to venv
# and run it there and seems to be all good.
print("Hello")

In [ ]:
# This cell allows hot reloading of modules.
# It is useful for development, so you don't have to restart the kernel.
# Some changes such as adding new files or changing the structure of the code
# may not be picked up and will require a kernel restart.
%load_ext autoreload
%autoreload 2

# Testing LINZ with Arcgis  
This notebook is for testing a python environment that has the ArcGIS API for Python installed, using LINZ as the testing portal.  

This notebook expects arcgis and not the geopandas module. This will force the data types to use the Spatially Enabled DataFrame (sdf).  

In [ ]:
import os
import json
import time
from dotenv import load_dotenv, find_dotenv
from importlib.metadata import version, PackageNotFoundError
import logging

# Set up logging include logger name
logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(name)s - %(levelname)s - %(message)s"
)
logger = logging.getLogger(__name__)

from kapipy.gis import GISK
from kapipy.gis import has_geopandas, has_arcgis, has_arcpy

if has_arcgis:
    import pandas as pd
    from arcgis.features import GeoAccessor, GeoSeriesAccessor
    from arcgis.gis import GIS

# find .env automagically by walking up directories until it's found
dotenv_path = find_dotenv()
load_dotenv(dotenv_path)
api_key = os.getenv("LINZ_API_KEY")

output_folder = r"c:\temp\data"

# layers
rail_station_layer_id = "50318"  # rail station 175 points
fences_layer_id = "50268"  # NZ Fence Centrelines
geodetic_marks_layer_id = "50787"  # NZ Geodetic Marks 132,966 point features
nz_walking_biking_tracks_layer_id = (
    "52100"  # NZ Walking and Biking Tracks 29,187 polyline features
)

# tables
nz_building_age_table_id = "105617"  # NZ Building Age 18 records, no changeset
suburb_locality_table_id = "113761"  # NZ Suburb Locality 3190 records

package_name = "kapipy"

try:
    pkg_version = version(package_name)
    print(f"{package_name} version: {pkg_version}")
except PackageNotFoundError:
    print(f"{package_name} is not installed")

print(f"{has_arcgis=}")
print(f"{has_arcpy=}")
print(f"{has_geopandas=}")

assert has_geopandas == False
assert has_arcgis == True

## Server object  
Get a reference to the LINZ server  

In [ ]:
linz = GISK(name="linz", api_key=api_key)
print(linz)

In [ ]:
#Audit Manager
linz.audit.enable_auditing(folder=r"c:/temp/audit")

## Crop Layers  

In [ ]:
linz.content.crop_layers.all()

In [ ]:
cl = linz.content.crop_layers.get(3036)
cl

In [ ]:
cl.filter_items("thames")

In [ ]:
cf = cl.get(10870 )
feature = cf.get()
feature.sdf

## Layer item  
Run some tests on a layer item  

In [ ]:
itm = linz.content.get(geodetic_marks_layer_id)
print(itm)

In [ ]:
print(f"{itm.supports_changesets=}")
print(f"{itm.data.fields=}")
print(f"{itm.data.crs=}")
print(f"{itm.data.geometry_type=}")
print(f"{itm.data.geometry_field=}")

In [ ]:
map = GIS().map("New Zealand")
map

In [ ]:
# read shapefile into an SDF
matamata_sdf = pd.DataFrame.spatial.from_featureclass("../tests/spatial_data/matamata_piako.shp")
matamata_sdf.spatial.plot(map_widget=map)
matamata_sdf.dtypes

In [ ]:
generalized_geoms = matamata_sdf[matamata_sdf.spatial.name].apply(lambda geom: geom.generalize(100))
matamata_sdf.spatial.set_geometry(generalized_geoms, inplace=True)
matamata_sdf.spatial.plot(map_widget=map)
matamata_sdf.dtypes


In [ ]:
p = matamata_sdf.spatial.bbox
print(type(p))
print(p.spatial_reference)
#p = p.project_as({"wkid": 4326})
p = p.project_as(4326)
print(p.spatial_reference)

In [ ]:
# Query using WFS
data = itm.query(
    count=100,
    out_sr=2193,
    filter_geometry=matamata_sdf,
    cql_filter="mark_condition='Reliably Placed'",
)
# res = itm.query(count=100, out_sr = 2193, filter_geometry=matamata_sdf)
# res = itm.query(count=100, out_sr = 2193, bbox=matamata_sdf)

logger.info(f"Total records returned {itm.title}: {data.sdf.shape[0]}")

data.sdf.spatial.plot(map_widget=map)
data.sdf.head()

In [ ]:
# Get a changeset from the layer using WFS
# cs_data = itm.changeset(from_time="2024-01-01T00:00:00Z", out_sr=2193)
cs_data = itm.changeset(
    from_time="2024-01-01T00:00:00Z",
    out_sr=2193,
    filter_geometry=matamata_sdf,
    cql_filter="mark_condition='Reliably Placed'",
)
logging.info(f"Total records returned {itm.title}: {cs_data.sdf.shape[0]}")
cs_data.sdf.head()

In [ ]:
cs_data.sdf.spatial.to_featureclass(os.path.join(output_folder, "sdf_changeset.shp"))

## Export item  

In [ ]:
#job = itm.export("geodatabase", out_sr=2193, filter_geometry=matamata_sdf.spatial.bbox)
job = itm.export("geodatabase", out_sr=2193, bbox_geometry=matamata_sdf)

In [ ]:
print(job.status)

In [ ]:
dl = job.download(folder=output_folder)

In [ ]:
print(f"{dl.folder=}")
print(f"{dl.filename=}")
print(f"{dl.file_path=}")
print(f"{dl.file_size_bytes=}")
print(f"{dl.download_url=}")
print(f"{dl.final_url=}")
print(f"{dl.job_id=}")
print(f"{dl.completed_at=}")
print(f"{dl.checksum=}")

## Table item  
Run some tests on a table item

In [ ]:
table_itm = linz.content.get(suburb_locality_table_id)
print(table_itm)

In [ ]:
print(f"{table_itm.supports_changesets=}")
print(f"{table_itm.data.fields=}")

In [ ]:
# Query using WFS to get DataFrame
data = table_itm.query(count=100)
logging.info(f"Total records returned {table_itm.title}: {data.df.shape[0]}")
data.df.head()

In [ ]:
# Query using WFS to get raw JSON
data = table_itm.query(count=10)
logging.info(data.json)
logging.info(f"Total records returned {table_itm.title}: {len(data.json.get('features'))}")


In [ ]:
# Get a changeset from the layer using WFS
cs_data = table_itm.changeset(from_time="2024-01-01T00:00:00Z", count=100)
logging.info(f"Total records returned, JSON count, {table_itm.title}: {len(cs_data.json.get('features'))}")
logging.info(f"Total records returned {table_itm.title}: {cs_data.df.shape[0]}")
cs_data.df.head()

In [ ]:
job = table_itm.export("geodatabase")

In [ ]:
print(job.status)

In [ ]:
job.download(folder=output_folder)

## Test multidownload samples  
Test downloading multiple jobs using the ContentManager download method.    

In [ ]:
itm1 = linz.content.get(geodetic_marks_layer_id)
itm2 = linz.content.get(rail_station_layer_id)
print(itm1)
print(itm2)

In [ ]:
job1 = itm1.export(
    "geodatabase",
    out_sr=2193,
    filter_geometry=matamata_sdf,
)
job2 = itm2.export(
    "geodatabase",
    out_sr=2193,
    filter_geometry=matamata_sdf,
)
results = linz.content.download(jobs=[job1, job2], folder=output_folder)

In [ ]:
for result in results:
    print(result.download_file_path)